In [ ]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import numpy as np
import json
from google.colab import files
import pickle

In [ ]:
from threading import Thread
from multiprocessing import Pool
from functools import partial
from itertools import repeat
from multiprocessing import Pool, freeze_support
from time import sleep

In [ ]:
cities=["Delhi-NCR","Chennai","Bengaluru","Mumbai","Pune","Hyderabad","Ahmedabad","Jaipur","Kolkata","Chandigarh","Mohali","Zirakpur","Udaipur","Ludhiana","Amritsar","Nagpur","Lucknow","Sonipat","Nasik","Surat","Vadodara","Aurangabad"]

In [ ]:
headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36"}

In [ ]:
def parse_links(links,city):
  data=[]
  response = get("https://www.cars24.com"+links,headers=headers)
  sleep(2)
  try:
    if response.status_code == 200:
      #print("ert")
      soup=BeautifulSoup(response.text,"html.parser")
      data.append(soup.find("h1").text)
      a=soup.find("div",attrs={"class":"_3Hvpt"})
      data.append(a.find("h2",attrs={"class":"U35sf"}).find("strong").text)
      a=soup.find("ul",attrs={"class":"_1wIhE"})
      b=a.find_all("p")
      l=a.find_all("label")
      labels_d={}
      for i in range(len(b)):
          if l[i].text in ["Year","Kms Driven","Fuel Type","Owner","Transmission"]:
              labels_d.update({l[i].text:b[i].text})
      for i in ["Year","Kms Driven","Fuel Type","Owner","Transmission"]:
          if i in labels_d.keys():
            data.append(labels_d[i])
          else:
            data.append(np.nan)
      data.append(city)
      #print(data)
  except:
      print("error")
  return data

In [ ]:
cities=[i.lower() for i in cities]

In [ ]:
car_name=[]
links={i:[] for i in cities}
for c in cities:
  print(c)
  base_url ="https://www.cars24.com/buy-used-cars-"+c+"/"
  response = get(base_url,headers=headers)
  soup=BeautifulSoup(response.text,"html.parser")
  no_of_pages=soup.find("div",attrs={"class":"col-sm-12 col-md-6 col-lg-7 col-xl-8"}).text
  no_of_pages=round(int(no_of_pages.split()[0])/21)
  print(no_of_pages)
  content=[]
  for i in range(1,no_of_pages+1):
    response = get(base_url+"?page="+str(i),headers=headers)
    soup=BeautifulSoup(response.text,"html.parser")
    content.extend(soup.find_all("div",attrs={"class":"_1gEgO"}))
  print("links-started")
  for j in content:
    car_name.append(j.find("h3").text)
    links[c].append(j.find("a").get("href"))
  print(len(links[c]))
  print(links[c][0])

delhi-ncr
1391
links-started
10000
/buy-used-Hyundai-Santro-Xing-2005-cars-New-Delhi-1005377985/
chennai
88
links-started
1760
/buy-used-Maruti-Wagon-R-2012-cars-Chennai-1134632085/
bengaluru
214
links-started
4280
/buy-used-Maruti-Alto-K10-2012-cars-Bengaluru-1025152488/
mumbai
256
links-started
5120
/buy-used-Maruti-Alto-2006-cars-Mumbai-1039860485/
pune
193
links-started
3860
/buy-used-Maruti-Alto-800-2015-cars-Pune-1062242988/
hyderabad
96
links-started
1920
/buy-used-Volkswagen-Ameo-2019-cars-Hyderabad-1042160789/
ahmedabad
265
links-started
5300
/buy-used-Hyundai-Grand-i10-2016-cars-Ahmedabad-1141770583/
jaipur
112
links-started
2240
/buy-used-Hyundai-NEW-SANTRO-2020-cars-Jaipur-1077890885/
kolkata
92
links-started
1840
/buy-used-Hyundai-Santro-Xing-2008-cars-Kolkata-1197246087/
chandigarh
194
links-started
3880
/buy-used-Maruti-Baleno-2016-cars-Chandigarh-1160546089/
mohali
45
links-started
900
/buy-used-Ford-Fiesta-2007-cars-Mohali-1095656589/
zirakpur
51
links-started
1020
/bu

In [ ]:
sum=0
for i in links.keys():
  sum+=len(links[i])
print(sum)

55320


In [ ]:
len(car_name)

55320

In [ ]:
cities

['delhi-ncr',
 'chennai',
 'bengaluru',
 'mumbai',
 'pune',
 'hyderabad',
 'ahmedabad',
 'jaipur',
 'kolkata',
 'chandigarh',
 'mohali',
 'zirakpur',
 'udaipur',
 'ludhiana',
 'amritsar',
 'nagpur',
 'lucknow',
 'sonipat',
 'nasik',
 'surat',
 'vadodara',
 'aurangabad']

In [ ]:
rec=[]
for i in list(links.keys())[1:]:
  print(i)
  with Pool(20) as p:
      rec.extend(p.starmap(parse_links,zip(links[i],repeat(i))))

In [ ]:
pd.DataFrame(rec)[7].value_counts()

ahmedabad     5040
mumbai        4819
pune          3940
chandigarh    3680
surat         2478
jaipur        2158
lucknow       2036
kolkata       1820
hyderabad     1799
vadodara      1740
ludhiana      1720
nagpur        1317
aurangabad     940
zirakpur       940
nasik          940
mohali         860
sonipat        800
amritsar       520
udaipur        360
Name: 7, dtype: int64

In [ ]:
len(links["delhi-ncr"]),len(records)

(10000, 1600)

In [ ]:
df_car=pd.DataFrame(rec)
#df_car["Car_name"]=car_name[:14420]

In [ ]:
df_car.columns=["variant","price","Year","Kms Driven","Fuel Type","Owner","Transmission","city"]

In [ ]:
index_to_be_dropped=df_car[df_car["variant"].isnull()].index

In [ ]:
new_df_car=df_car.drop(index=index_to_be_dropped)

In [ ]:
new_df_car["car_name"]=new_df_car["variant"].apply(lambda x:x.split()[0]+" "+x.split()[1])

In [ ]:
new_df_car.head()

,variant,price,Year,Kms Driven,Fuel Type,Owner,Transmission,city,car_name
0,Hyundai i20 ASTA 1.2,"160,200",2011,"69,830",Petrol + CNG,Third,Manual,mumbai,Hyundai i20
1,Honda City SV MT PETROL,"615,593",2016,"39,477",Petrol,First,Manual,mumbai,Honda City
2,Maruti Wagon R 1.0 LXI CNG,"238,882",2011,"58,778",Petrol + CNG,First,Manual,mumbai,Maruti Wagon
3,Ford Ecosport 1.5AMBIENTE TI VCT,"487,340",2014,"39,496",Petrol,First,Manual,mumbai,Ford Ecosport
4,Maruti Baleno ALPHA 1.2 K12,"611,390",2017,"45,694",Petrol,First,Manual,mumbai,Maruti Baleno


In [ ]:
new_df_car.to_csv("remaining_cars24.csv",index=False)
files.download("remaining_cars24.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
che=pd.read_csv("/content/drive/My Drive/car_project/chennai_24cars.csv")
del_ban=pd.read_csv("/content/drive/My Drive/car_project/bangalore_delhi_24cars.csv")
rem=pd.read_csv("/content/drive/My Drive/car_project/remaining_cars24.csv")

In [ ]:
che.head()

,variant,price,Year,Kms Driven,Fuel Type,Owner,Transmission,city,car_name
0,Maruti Swift VDI ABS,"474,000",2016,"74,764",Diesel,First,Manual,Chennai,Maruti Swift
1,Maruti Celerio VXI,"262,000",2014,"89,660",Petrol,First,Manual,Chennai,Maruti Celerio
2,Hyundai i10 SPORTZ 1.2 AT KAPPA2,"225,000",2010,"73,801",Petrol,First,Automatic,Chennai,Hyundai i10
3,Nissan Micra XE DIESEL,"300,000",2013,"72,583",Diesel,Second,Manual,Chennai,Nissan Micra
4,Fiat Grand Punto ACTIVE 1.3,"195,000",2012,"63,271",Diesel,Second,Manual,Chennai,Fiat Grand Punto


In [ ]:
del_ban.head()

,variant,price,Year,Kms Driven,Fuel Type,Owner,Transmission,city,car_name
0,Maruti Ertiga ZDI,"454,675",2012,"98,284",Diesel,First,Manual,delhi-ncr,Maruti Ertiga
1,Maruti Ciaz VXI PLUS,"658,520",2015,"11,600",Petrol,First,Manual,delhi-ncr,Maruti Ciaz
2,Maruti Swift VDI,"220,366",2011,"75,829",Diesel,First,Manual,delhi-ncr,Maruti Swift
3,Maruti Ertiga ZDI,"454,675",2012,"98,284",Diesel,First,Manual,delhi-ncr,Maruti Ertiga
4,Maruti Swift Dzire VXI 1.3,"257,084",2009,"75,753",Petrol,First,Manual,delhi-ncr,Maruti Swift Dzire


In [ ]:
rem.head()

,variant,price,Year,Kms Driven,Fuel Type,Owner,Transmission,city,car_name
0,Hyundai i20 ASTA 1.2,"160,200",2011,"69,830",Petrol + CNG,Third,Manual,mumbai,Hyundai i20
1,Honda City SV MT PETROL,"615,593",2016,"39,477",Petrol,First,Manual,mumbai,Honda City
2,Maruti Wagon R 1.0 LXI CNG,"238,882",2011,"58,778",Petrol + CNG,First,Manual,mumbai,Maruti Wagon
3,Ford Ecosport 1.5AMBIENTE TI VCT,"487,340",2014,"39,496",Petrol,First,Manual,mumbai,Ford Ecosport
4,Maruti Baleno ALPHA 1.2 K12,"611,390",2017,"45,694",Petrol,First,Manual,mumbai,Maruti Baleno


In [ ]:
final_df=pd.concat([che,del_ban,rem])

In [ ]:
new_df_car.to_csv("final_cars24.csv",index=False)
files.download("final_cars24.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>